In [1]:
import spacy
import nltk
import pandas as pd
import re
import joblib
import math
import os
import string
import warnings
warnings.filterwarnings("ignore")

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [3]:
from transformers import pipeline, RobertaTokenizerFast, TFRobertaForSequenceClassification

tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
emotion_pipeline = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
# Initializing NLTK libraries
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to remove html links
def clean_html(html):
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script', 'code', 'a']):
        data.decompose()
    return ' '.join(soup.stripped_strings)

# Function to clean and lemmatize the string
def clean_string(text, stem='Spacy'):
    lemmatizer = WordNetLemmatizer()
    clean_text = re.sub(r'\s+', ' ', text)
    clean_text = ' '.join([lemmatizer.lemmatize(word) for word in clean_text.split() if word not in stopwords.words('english')])
    return clean_text

In [25]:
text = '''Dear Staffenberg,

I hope this email finds you in good health and high spirits. Today, I want to take a moment to reflect on the beauty of life and share some thoughts that have been resonating with me lately.

Life is an intricate tapestry of experiences, emotions, and connections. It is a journey filled with ups and downs, twists and turns, and unexpected surprises. It's easy to get caught up in the daily grind and lose sight of the wonder that surrounds us. However, it is in these moments of reflection that we can truly appreciate the magnificence of life.

Think about the simple pleasures that bring you joy: a warm cup of coffee on a chilly morning, the gentle touch of a loved one, or the breathtaking colors of a sunset. These seemingly ordinary moments are what make life extraordinary. By cultivating an attitude of gratitude, we can find beauty and fulfillment in even the smallest things.

Life is also about growth and learning. Each experience, whether pleasant or challenging, offers an opportunity for personal development. It is through adversity that we discover our strength and resilience. Embracing change and stepping out of our comfort zones can lead to profound transformations and new horizons. Remember, it's not about the destination but the journey itself.

Moreover, life is about connections and relationships. Our interactions with others shape our experiences and contribute to our sense of belonging. Take a moment to cherish the people who have touched your life, be it family, friends, or even strangers who have shown kindness. Nurture these relationships, for they provide support, understanding, and shared moments of laughter and tears.

While life can be unpredictable and challenging, it is essential to maintain a sense of hope and optimism. Embrace the power of dreams and aspirations, for they fuel our passions and drive us forward. Celebrate your achievements, no matter how small, and use setbacks as stepping stones to greater success.

In conclusion, life is a precious gift, and it is up to us to make the most of it. Let us approach each day with gratitude, curiosity, and an open heart. Embrace the beauty that surrounds you, seek personal growth, nurture relationships, and hold onto hope. Remember, the true essence of life lies not in its length but in the depth of our experiences.

May you find joy, fulfillment, and meaning in every step of your journey.

Warm regards,
Kyle'''

In [26]:
text = re.sub(r'[^\w\s]', '', text)
clean_text = clean_html(text)
processed_text = clean_string(clean_text)

In [31]:
temp_list = processed_text.split()
emotion_lists = {
    'admiration': [], 'amusement': [], 'anger': [], 'annoyance': [],
    'approval': [], 'caring': [], 'confusion': [], 'curiosity': [],
    'desire': [], 'disappointment': [], 'disapproval': [], 'disgust': [],
    'embarrassment': [], 'excitement': [], 'fear': [], 'gratitude': [],
    'grief': [], 'joy': [], 'love': [], 'nervousness': [], 'optimism': [],
    'pride': [], 'realization': [], 'relief': [], 'remorse': [], 'sadness': [],
    'surprise': []
}

for x in temp_list:
    results = emotion_pipeline(x)
    if results != 'neutral':
        for result in results:
            top_label = max(result, key=lambda dictionary: dictionary['score'])
            label = top_label["label"]
            if label in emotion_lists:
                emotion_lists[label].append(x)
            
print(emotion_lists)

{'admiration': ['good', 'beauty', 'wonder', 'truly', 'appreciate', 'magnificence', 'breathtaking', 'beauty', 'kindness', 'greater', 'precious', 'beauty'], 'amusement': ['laughter'], 'anger': [], 'annoyance': [], 'approval': ['warm'], 'caring': [], 'confusion': [], 'curiosity': ['curiosity'], 'desire': ['want'], 'disappointment': ['lose', 'seemingly'], 'disapproval': [], 'disgust': [], 'embarrassment': ['Embracing'], 'excitement': [], 'fear': ['stranger'], 'gratitude': ['Dear', 'success'], 'grief': [], 'joy': ['pleasure', 'joy', 'coffee', 'gentle', 'pleasant', 'Celebrate', 'joy'], 'love': ['loved'], 'nervousness': [], 'optimism': ['hope', 'hope', 'hope'], 'pride': [], 'realization': [], 'relief': [], 'remorse': [], 'sadness': ['lately'], 'surprise': ['unexpected', 'surprise', 'extraordinary']}


In [38]:
res = {key: val for key, val in emotion_lists.items() if val}
print(res)


{'admiration': ['good', 'beauty', 'wonder', 'truly', 'appreciate', 'magnificence', 'breathtaking', 'beauty', 'kindness', 'greater', 'precious', 'beauty'], 'amusement': ['laughter'], 'approval': ['warm'], 'curiosity': ['curiosity'], 'desire': ['want'], 'disappointment': ['lose', 'seemingly'], 'embarrassment': ['Embracing'], 'fear': ['stranger'], 'gratitude': ['Dear', 'success'], 'joy': ['pleasure', 'joy', 'coffee', 'gentle', 'pleasant', 'Celebrate', 'joy'], 'love': ['loved'], 'optimism': ['hope', 'hope', 'hope'], 'sadness': ['lately'], 'surprise': ['unexpected', 'surprise', 'extraordinary']}


In [40]:
for emotion, words in res.items():
    print(f"Emotion: {emotion}")
    print("Words:")
    for word in words:
        print(word)
    print()

Emotion: admiration
Words:
good
beauty
wonder
truly
appreciate
magnificence
breathtaking
beauty
kindness
greater
precious
beauty

Emotion: amusement
Words:
laughter

Emotion: approval
Words:
warm

Emotion: curiosity
Words:
curiosity

Emotion: desire
Words:
want

Emotion: disappointment
Words:
lose
seemingly

Emotion: embarrassment
Words:
Embracing

Emotion: fear
Words:
stranger

Emotion: gratitude
Words:
Dear
success

Emotion: joy
Words:
pleasure
joy
coffee
gentle
pleasant
Celebrate
joy

Emotion: love
Words:
loved

Emotion: optimism
Words:
hope
hope
hope

Emotion: sadness
Words:
lately

Emotion: surprise
Words:
unexpected
surprise
extraordinary

